<a href="https://colab.research.google.com/github/AhmedFarrukh/DeepLearning-EdgeComputing/blob/main/notebooks/MeasuringInferenceTimes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, the inference times and memory footprint of the original and quantized versions of 7 popular Convolutional Neural Networks are measured and compared.

The CNN models are: MobileNet, InceptionV3, Resnet50, ResNet101, ResNet152, VGG16, VGG19.

The quantized models were created by applying [Post-training Dynamic Range Quantization](https://www.tensorflow.org/lite/performance/post_training_quantization).

Both the original models, and their quantized versions, are of tflite format, and were uploaded to [Google Drive](https://drive.google.com/drive/folders/1OcJ9ceYg6ZWFJ4QMR0zznsw0KVeHPa4h?usp=drive_link).

The benchmarking of models is achieved by using the official [TFlite benchmark](https://www.tensorflow.org/lite/performance/measurement) which measures the following metrics:
*   Initialization time
*   Inference time of warmup state
*   Inference time of steady state
*   Memory usage during initialization time
*   Overall memory usage

The benchmark generates a series of random inputs, runs the models and aggregates the results to report the aforementioned metrics.

In [1]:
modelNames = ["MobileNet", "InceptionV3", "ResNet50", "ResNet101", "ResNet152", "VGG16", "VGG19"]

We can download the models from the Google Drive using gdown. If you want to download your own set of models, you can modify the google drive link below. In this case, we download the models to the /root/tflite_models directory.

In [18]:
%cd ..
import gdown
gdown.download_folder('https://drive.google.com/drive/folders/1OcJ9ceYg6ZWFJ4QMR0zznsw0KVeHPa4h')

/home/cc/DeepLearning-EdgeComputing


Retrieving folder contents


Processing file 190OgRXUl3gZfxLqqayk6ebW0Eyu6zuii InceptionV3_quant.tflite
Processing file 1qHe6rtJgsnS_JMa-XK5160DHU2wICrD_ InceptionV3.tflite
Processing file 12uKNdkCGtPnKr5nYDOG8RiU2r8cHlP5U MobileNet_quant.tflite
Processing file 1pjsdyJdO7JKz1Qc1q3Hypd_JSKSTCrO6 MobileNet.tflite
Processing file 1-3jO54_5e6E22wG-P_Km-SHMxLlUnT_r ResNet50_quant.tflite
Processing file 1cl1g9EN307xLkxH-12IVJ76x-kobeMhf ResNet50.tflite
Processing file 1oCjM_VWXrAVToRwuUFgW1bupBpmc7Rwx ResNet101_quant.tflite
Processing file 1yImWscZUsOANKwV15WIN77o7Q2Yrkc14 ResNet101.tflite
Processing file 1824S3uNOro0WBlccpflEJhXzGR8k7L5w ResNet152_quant.tflite
Processing file 1JwfKL79O75XiamF5iEqGkUSt9LYygwS3 ResNet152.tflite
Processing file 1m-RI9IV3vIuiHJ-BoHc4XnPn0fewJzGs VGG16_quant.tflite
Processing file 1XywqHW8wmqkfTw6cjo7xlnYaTlr9AbwE VGG16.tflite
Processing file 1Jo-SqyPZvCG8p2Cvpq1tNPhd3LavQ24_ VGG19_quant.tflite
Processing file 1sEfBpOdcw-LQAl24Y0SMH4fVWoyCCBki VGG19.tflite


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=190OgRXUl3gZfxLqqayk6ebW0Eyu6zuii
To: /home/cc/DeepLearning-EdgeComputing/tflite_models/InceptionV3_quant.tflite
100%|██████████| 24.1M/24.1M [00:00<00:00, 70.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qHe6rtJgsnS_JMa-XK5160DHU2wICrD_
To: /home/cc/DeepLearning-EdgeComputing/tflite_models/InceptionV3.tflite
100%|██████████| 95.3M/95.3M [00:02<00:00, 46.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=12uKNdkCGtPnKr5nYDOG8RiU2r8cHlP5U
To: /home/cc/DeepLearning-EdgeComputing/tflite_models/MobileNet_quant.tflite
100%|██████████| 4.42M/4.42M [00:00<00:00, 23.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pjsdyJdO7JKz1Qc1q3Hypd_JSKSTCrO6
To: /home/cc/DeepLearning-EdgeComputing/tflite_models/MobileNet.tflite
100%|██████████| 16.9M/16.9M [00:00<00:00, 49.0MB/s]
Downloading...
From: https://drive.goog

['/home/cc/DeepLearning-EdgeComputing/tflite_models/InceptionV3_quant.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/InceptionV3.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/MobileNet_quant.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/MobileNet.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/ResNet50_quant.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/ResNet50.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/ResNet101_quant.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/ResNet101.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/ResNet152_quant.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/ResNet152.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/VGG16_quant.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/VGG16.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_models/VGG19_quant.tflite',
 '/home/cc/DeepLearning-EdgeComputing/tflite_

You can verify that the models were correctly loaded by listing the files in the /root/tflite_models directory. Note that there should be two tflite files for each model: an original and a quantized version. The size of the quantized models should be significantly smaller than the size of their corresponding original model.

In [19]:
!ls -lR /home/cc/DeepLearning-EdgeComputing/tflite_models

/home/cc/DeepLearning-EdgeComputing/tflite_models:
total 2152992
-rw-rw-r-- 1 cc cc  95324856 Jul 21 15:40 InceptionV3.tflite
-rw-rw-r-- 1 cc cc  24138200 Jul 21 15:43 InceptionV3_quant.tflite
-rw-rw-r-- 1 cc cc  16903376 Jul 21 15:43 MobileNet.tflite
-rw-rw-r-- 1 cc cc   4423744 Jul 21 15:43 MobileNet_quant.tflite
-rw-rw-r-- 1 cc cc 178055132 Jul 21 15:25 ResNet101.tflite
-rw-rw-r-- 1 cc cc  45360928 Jul 21 15:42 ResNet101_quant.tflite
-rw-rw-r-- 1 cc cc 240570616 Jul 21 15:16 ResNet152.tflite
-rw-rw-r-- 1 cc cc  61361208 Jul 21 15:41 ResNet152_quant.tflite
-rw-rw-r-- 1 cc cc 102157452 Jul 21 15:39 ResNet50.tflite
-rw-rw-r-- 1 cc cc  25968784 Jul 21 15:43 ResNet50_quant.tflite
-rw-rw-r-- 1 cc cc 553439904 Jul 21 15:05 VGG16.tflite
-rw-rw-r-- 1 cc cc 138458672 Jul 21 15:37 VGG16_quant.tflite
-rw-rw-r-- 1 cc cc 574680200 Jul 21 14:41 VGG19.tflite
-rw-rw-r-- 1 cc cc 143789152 Jul 21 15:31 VGG19_quant.tflite


Next, we download the TFlite benchmark which we will use to measure inference times and memory footprint. More details about the benchmark can be found on the [tensorflow website](https://www.tensorflow.org/lite/performance/measurement). Note that the benchmark is specific to the architecture type (such as x86 or ARM), and the appropriate benchmark binary must be downloaded. Below, the benchmark is loaded for an x86-64 type architecture.

The benchmark is downloaded to the /root/benchmark folder, and its permissions are then updated to allow it to be executed.

In [20]:
!mkdir /home/cc/DeepLearning-EdgeComputing/benchmark
!wget https://storage.googleapis.com/tensorflow-nightly-public/prod/tensorflow/release/lite/tools/nightly/latest/linux_x86-64_benchmark_model -P /home/cc/DeepLearning-EdgeComputing/benchmark
!chmod +x /home/cc/DeepLearning-EdgeComputing/benchmark/linux_x86-64_benchmark_model

--2024-07-25 11:55:50--  https://storage.googleapis.com/tensorflow-nightly-public/prod/tensorflow/release/lite/tools/nightly/latest/linux_x86-64_benchmark_model
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.207, 64.233.180.207, 172.253.63.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6205384 (5.9M) [application/octet-stream]
Saving to: ‘/home/cc/DeepLearning-EdgeComputing/benchmark/linux_x86-64_benchmark_model’

linux_x86-64_benchm 100%[===================>]   5.92M  28.3MB/s    in 0.2s    

2024-07-25 11:55:50 (28.3 MB/s) - ‘/home/cc/DeepLearning-EdgeComputing/benchmark/linux_x86-64_benchmark_model’ saved [6205384/6205384]



Let's run the benchmark on the MobileNet_quant model and note the output.

In [21]:
!/home/cc/DeepLearning-EdgeComputing/benchmark/linux_x86-64_benchmark_model \
      --graph=/home/cc/DeepLearning-EdgeComputing/tflite_models/MobileNet_quant.tflite \
      --num_threads=1

INFO: STARTING!
INFO: Log parameter values verbosely: [0]
INFO: Num threads: [1]
INFO: Graph: [/home/cc/DeepLearning-EdgeComputing/tflite_models/MobileNet_quant.tflite]
INFO: Signature to run: []
INFO: #threads used for CPU inference: [1]
INFO: Loaded model /home/cc/DeepLearning-EdgeComputing/tflite_models/MobileNet_quant.tflite
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: The input model file size (MB): 4.42374
INFO: Initialized session in 44.168ms.
INFO: Running benchmark for at least 1 iterations and at least 0.5 seconds but terminate if exceeding 150 seconds.
INFO: count=60 first=9810 curr=8562 min=8267 max=9810 avg=8401.87 std=215

INFO: Running benchmark for at least 50 iterations and at least 1 seconds but terminate if exceeding 150 seconds.
INFO: count=119 first=8363 curr=8315 min=8262 max=8739 avg=8354.76 std=77

INFO: Inference timings in us: Init: 44168, First inference: 9810, Warmup (avg): 8401.87, Inference (avg): 8354.76
INFO: Note: as the benchmark tool 

Since the result of the benchmark is reported as text on the console, we can define a parsing function to extract the data. The parsing function takes the output of the benchmark as an input and adds the results to a dictionary of metrics.

In [22]:
import re

def parse_benchmark_output(output, results):
    """
    Parse benchmark output to extract model initialization times, inference timings, and memory footprint.
    """

    # Regular expressions to match the required information
    init_time_patterns = [
        re.compile(r'INFO: Initialized session in (\d+.\d+)ms.'),
        re.compile(r'INFO: Initialized session in (\d+)ms.')
    ]
    inference_patterns = [
        re.compile(r'INFO: Inference timings in us: Init: (\d+), First inference: (\d+), Warmup \(avg\): (\d+.\d+), Inference \(avg\): (\d+.\d+)'),
        re.compile(r'INFO: Inference timings in us: Init: (\d+), First inference: (\d+), Warmup \(avg\): (\d+), Inference \(avg\): (\d+)'),
        re.compile(r'INFO: Inference timings in us: Init: (\d+), First inference: (\d+), Warmup \(avg\): ([\d.e+]+), Inference \(avg\): (\d+)')
    ]
    memory_pattern = re.compile(r'INFO: Memory footprint delta from the start of the tool \(MB\): init=(\d+.\d+) overall=(\d+.\d+)')

    for line in output.split('\n'):
        # Match the initialization time
        for pattern in init_time_patterns:
            init_match = pattern.search(line)
            if init_match:
                results['Init Time (ms)'].append(float(init_match.group(1)))
                break

        # Match the inference timings
        for pattern in inference_patterns:
            inference_match = pattern.search(line)
            if inference_match:
                results["Init Inference (ms)"].append(int(inference_match.group(1))/1000)
                results["First Inference (ms)"].append(int(inference_match.group(2))/1000)
                results["Warmup Inference (ms)"].append(float(inference_match.group(3))/1000)
                results["Avg Inference (ms)"].append(float(inference_match.group(4))/1000)
                break

        # Match the memory footprint
        memory_match = memory_pattern.search(line)
        if memory_match:
            results['Memory Init (MB)'].append(float(memory_match.group(1)))
            results['Memory Overall (MB)'].append(float(memory_match.group(2)))



Next, we can define a Pandas Dataframe to store our results. Since we will be repeatedly running the benchmark to estimate the standard deviation of results as well, for each metric, we will define two columns - one for the mean and the other for the standard deviation.

In [23]:
import pandas as pd

# Define model types (rows)
rows = []
for model in modelNames:
  rows.append(model)
  rows.append(model + "_quant")

# Define features (columns)
cols = []
features = ["Init Time (ms)", "Init Inference (ms)", "First Inference (ms)", "Warmup Inference (ms)", "Avg Inference (ms)", "Memory Init (MB)", "Memory Overall (MB)"]
for feature in features:
  cols.append(feature)
  cols.append(feature + "_sd")

# Create an empty DataFrame
finalResult = pd.DataFrame(index=rows, columns=cols)

Finally, run the benchmark repeatedly and average the results. For each model, we repeatedly run the benchmark, and parse the output from the benchmark. After `n` trials, the mean and standard deviation of the metrics is added to the `finalResult` dataframe defined in the last step.

In [ ]:
import subprocess
from collections import defaultdict
from statistics import mean
from statistics import stdev

n = 10 #the number of times the benchmark is called for each model

for modelName in rows:
  print(modelName)
  modelResults = defaultdict(list)
  for i in range(n):
    outputOriginal = subprocess.check_output("/home/cc/DeepLearning-EdgeComputing/benchmark/linux_x86-64_benchmark_model \
      --graph=/home/cc/DeepLearning-EdgeComputing/tflite_models/" + modelName +".tflite"+" \
      --num_threads=1", shell=True)
    outputOriginal = outputOriginal.decode('utf-8')
    output = parse_benchmark_output(outputOriginal, modelResults)

  for feature in features:
    finalResult.loc[modelName, feature] = mean(modelResults[feature])
    finalResult.loc[modelName, feature + "_sd"] = stdev(modelResults[feature])

MobileNet


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


MobileNet_quant


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


InceptionV3


...

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


VGG16_quant


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


VGG19


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Let's have a look at the results.

In [ ]:
print(finalResult)

Finally, we can generate plots of the results.

Let's create a directory to store the results:/home/cc/DeepLearning-EdgeComputing/notebooks/results

In [ ]:
!mkdir /home/cc/DeepLearning-EdgeComputing/notebooks/results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
for feature in features:
    means_orig = finalResult.loc[modelNames, feature].values
    errors_orig = finalResult.loc[modelNames, feature + "_sd"].values
    means_quant = finalResult.loc[[model + "_quant" for model in modelNames], feature].values
    errors_quant = finalResult.loc[[model + "_quant" for model in modelNames], feature + "_sd"].values


    n_groups = len(modelNames)
    index = np.arange(n_groups)

    fig, ax = plt.subplots()
    bar_width = 0.35
    opacity = 0.8

    rects1 = plt.bar(index, means_orig, bar_width,
                     alpha=opacity,
                     yerr=errors_orig,
                     label='Original')

    rects2 = plt.bar(index + bar_width, means_quant, bar_width,
                     alpha=opacity,
                     yerr=errors_quant,
                     label='Quantized')

    plt.xlabel('Model')
    plt.ylabel(feature)
    plt.title(f'Bar Chart for {feature}')
    plt.xticks(index + bar_width / 2, modelNames, rotation=45)
    plt.legend()

    plt.tight_layout()
    plt.savefig(f'/home/cc/DeepLearning-EdgeComputing/notebooks/results/{feature}.png')
    # Show the plot
    plt.show()